# Лабораторная работа №3
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [55]:
# Необходим при замере скорости выполнения кода
from timeit import default_timer as timer
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [39]:
def get_suffix_length(substring: str):
    suf_lengths = [0]*len(substring)
    j = 0
    i = 1

    while i < len(substring):
        if substring[j] == substring[i]:
            suf_lengths[i] = j+1
            i += 1
            j += 1
        else:
            if j == 0:
                suf_lengths[i] = 0
                i += 1
            else:
                j = suf_lengths[j-1]
                
    return suf_lengths

In [41]:
def substring_kmp_index(substring: str, string: str):
    suf_length = get_suffix_length(substring)
    index_string = 0
    index_substring = 0

    while index_string < len(string):
        if string[index_string] == substring[index_substring]:
            index_string += 1
            index_substring += 1
            if index_string == len(substring):
                return index_string-index_substring
        else:
            if index_substring > 0:
                index_substring = suf_length[index_substring-1]
            else:
                index_string += 1
    
    return None

#### Упрощенный алгоритм Бойера-Мура

In [38]:
def get_offsets(substring: str):
    offsets = defaultdict(set)

    for i in range(len(substring)-2, -1, -1):
        if not offsets.__contains__(substring[i]):
            offsets.setdefault(substring[i], len(substring)-i-1)
            continue
        offsets.setdefault(substring[i], offsets[substring[i]])
    
    if not offsets.__contains__(substring[len(substring)-1]):
            offsets.setdefault(substring[len(substring)-1], len(substring))
    else:
        offsets.setdefault(substring[len(substring)-1], offsets[substring[len(substring)-1]])
    
    return offsets

In [47]:
def substring_bmh_index(substring: str, string: str):
    offsets = get_offsets(substring)

    current_offset = 1
    for start_index in range(len(substring)-1, len(string), current_offset):
        for j in range(len(substring)-1, -1, -1):

            if substring[j]!=string[start_index-(len(substring)-1)+j]:
                if j == 0:
                    if (offsets.__contains__(string[start_index-(len(substring)-1)+j])):
                        current_offset = offsets[substring[j]]
                        break
                    
                    current_offset = len(substring)
                    break

                current_offset = offsets[substring[j]]
                break

            if j == 0:
                return start_index-len(substring)+1
    return None

### Сравнение времени работы алгоритмов

In [45]:
def substring_standard_index(substring: str, string: str):
    return string.find(substring)

In [50]:
string = 'sdfjfjasdlk'
substring = 'fjas'

In [58]:
substring_searches = dict(
    kmp_substring = substring_kmp_index,
    bmh_substring = substring_bmh_index,
    standard_substring = substring_standard_index
)

In [59]:
for name, callable in substring_searches.items():
    start = timer()
    callable(substring, string)
    end = timer()
    print(f"{name}: {end-start}")

kmp_substring: 1.06999996205559e-05
bmh_substring: 1.1000000085914508e-05
standard_substring: 1.999998858082108e-06


На данном примере стандартный поиск показал себя наиболее эффективным, но из самописных поисков наибольшую эффективность показал алгоритм Кнута-Морриса-Пратта

### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

# короч головоломка-пазл с одной свободной ячейкой, где надо переставлять все остальные

### пятнашки

In [1]:
class Board:
    def __init__(self, cells: list[list[int]]):
        self.cells = self.__get_cells_copy(cells)
        self.zero_x = None
        self.zero_y = None
        self.h = 0

        for i in range(self.get_width()):
            for j in range (self.get_height()):
                if cells[i][j] != (i*self.get_height()+j+1) % self.get_size():
                    self.h+=1
                if cells[i][j]==0:
                    self.zero_x = i
                    self.zero_y = j

    def __eq__(self, other):
        if self is None:
            if other is None:
                return True
            return False
        if other is None:
            return False
        
        if self.get_width() != other.get_width() or self.get_height != other.get_height():
            return False

        for i in range(self.get_width()):
            for j in range(self.get_height()):
                if self.cells[i][j] != other.get_cell(i,j):
                    return False

        return True

    def __str__(self):
        result = ''
        max_length = len(str(self.get_size()-1))

        for i in range(self.get_width()):
            for j in range(self.get_height()):
                result += str(self.__get_cell(i,j)).rjust(max_length+1)
            result += '\n'
        
        return result

    def get_neighbors(self):
        boards_variants = list()
        result = list()
        
        boards_variants.append(self.__change_neighbor_points(self.zero_x, self.zero_y, self.zero_x - 1, self.zero_y))
        boards_variants.append(self.__change_neighbor_points(self.zero_x, self.zero_y, self.zero_x + 1, self.zero_y))
        boards_variants.append(self.__change_neighbor_points(self.zero_x, self.zero_y, self.zero_x, self.zero_y - 1))
        boards_variants.append(self.__change_neighbor_points(self.zero_x, self.zero_y, self.zero_x, self.zero_y + 1))
        
        for item in boards_variants:
            if item is not None:
                result.append(item)

        return result
        
    def __change_neighbor_points(self, x1: int, y1: int, x2: int, y2: int):
        if self.__is_coord_exists(x2, y2):
            new_cells = self.__get_cells_copy(self.cells)
            new_cells[x1][y1], new_cells[x2][y2] = new_cells[x2][y2], new_cells[x1][y1]
            return Board(new_cells)
        return None

    def get_list_of_cells(self):
        cells_list = list()
        for i in range(self.get_width()):
            for j in range (self.get_height()):
                cells_list.append(self.__get_cell(i,j))
        return cells_list

    def is_goal(self):
        return self.h == 0
    
    def get_h(self):
        return self.h
        
    def get_size(self):
        return self.get_width()*self.get_height()

    def get_width(self):
        if (self.cells != None):
            return len(self.cells)
        return None
    
    def get_height(self):
        if (self.cells != None):
            return len(self.cells[0])
        return None

    def __get_cell(self, x: int, y: int):
        if self.__is_coord_exists(x,y):
            return self.cells[x][y]
        return None

    def __get_cells_copy(self, start_cells):
        result = list()
        for element in start_cells:
            result.append(element.copy())
        return result

    def __is_coord_exists(self, x: int, y: int):
        return x in range(self.get_width()) and y in range(self.get_height())

In [87]:
class Solver:
    def __init__(self, board: Board):
        self.board = board
        self.current_item = self.Item(board)

    class Item:
        def __init__(self, current: Board, previous = None):
            self.current = current
            self.previous = previous
            self.g = 0

            if previous is not None:
                self.g = previous.get_g()+1

        def is_previous_eq(self, other):
            return self.previous == other

        def set_previous(self, new_previous):
            self.previous = new_previous

        def set_g(self, g):
            self.g = g

        def get_h(self):
            return self.get_board().get_h()

        def get_g(self):
            return self.g

        def get_f(self):
            return self.get_g() + self.get_h()

        def get_previous(self):
            return self.previous

        def get_board(self):
            return self.current
        
        def is_contains(self, other: Board):
            current = self
            while current is not None:
                if current.get_board() == other:
                    return True
                current = self.get_previous()
            return False

    def get_solve(self):
        #if self.__is_solvable():   
        if not self.parity():     
            open_states = list()
            closed_states = list()

            open_states.append(self.current_item)

            #TODO РЕШЕНИЕ С ПОМОЩЬЮ А*
            #print (current_item.get_board())
            while len(open_states) != 0:
                current_item = self.__get_item_with_min_f(open_states)

                if current_item.get_board().is_goal():
                    #print(current_item.get_board())
                    return self.__get_path(current_item)
                
                self.__remove_item_from_list(open_states, current_item)
                closed_states.append(current_item)
                #print(current_item.get_board())

                for neighbor in current_item.get_board().get_neighbors():
                    neighbor_item = self.Item(neighbor, current_item)
                    
                    if self.__is_list_contains_item(closed_states, neighbor_item):
                        continue
                    
                    if not self.__is_list_contains_item(open_states, neighbor_item):
                        open_states.append(neighbor_item)
                    
        return None

    def __is_list_contains_item(self, items_list, item):
        for i in range(len(items_list)):
            if items_list[i]==item:
                return i
        return None

    def __remove_item_from_list(self, items_list, item):
        index = self.__is_list_contains_item(items_list, item)
        if index is not None: 
            del items_list[index]


    def __get_item_with_min_f(self, items: list()):
        if len(items) != 0:
            result = items[0]
            for item in items:
                if item.get_f() < result.get_f():
                    result = item
            items.remove(result)
            
            return result
        return None

    def __is_solvable(self):
        parity_result = 0
        cells_list = self.board.get_list_of_cells()
        for i in range(len(cells_list)):
            if cells_list[i] == 0:
                parity_result += i//self.board.get_width()+1
                continue

            for j in range(i+1, len(cells_list)):
                if cells_list[i]>cells_list[j] and cells_list[j]!=0:
                    parity_result+=1 
        
        return parity_result % 2 == 0
    
    def parity(self):
        permutation = self.board.get_list_of_cells()
        # Создаем набор из уникальных данных с пройденными ячейками
        # и создаем кол-во циклов прохождения по массиву ячеек
        seen, cycles = set(), 0
        for i in permutation:
            # Проверяем на наличие текущей ячейки в нашем наборе при перестановке
            # и добавляем его в набор при его отсутствии в нем
            if i not in seen:
                # Наращиваем счетчик циклов при отсутствии значений в наборе
                cycles += 1
                while i not in seen:
                    seen.add(i)
                    i = permutation[i]
        return (cycles + len(permutation)) % 2 == 0

    def __get_path(self, item: Item):
        path = list()
        current = item
        path.append(current.get_board())
        while current.get_previous() is not None:
            current = current.get_previous()
            path.append(current.get_board())
        return path[::-1]


In [91]:
puzzle = [[1,2,3,4],[5,6,7,8],[13,9,11,12],[10,14,15,0]]

initial = Board(puzzle)
#print(initial.get_h())
#print()

solve = Solver(initial).get_solve()
if solve is None:
    print("Нет решения")
else:
    for element in solve:
        print(element)
#print(Solver(initial))

#print(solver.moves())

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14  0 15

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10  0 14 15

  1  2  3  4
  5  6  7  8
 13  9 11 12
  0 10 14 15

  1  2  3  4
  5  6  7  8
  0  9 11 12
 13 10 14 15

  1  2  3  4
  5  6  7  8
  9  0 11 12
 13 10 14 15

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13  0 14 15

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14  0 15

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15  0



### Вывод

Экспериментально было выяснено, что стандартный поиск показывает себя наиболее эффективным в сравнении с КМП и БМХ, но из самописных поисков наибольшую эффективность показал алгоритм Кнута-Морриса-Пратта. Также в ходе лабораторной работы была написана программа, находящая решение пятнашек